<a href="https://colab.research.google.com/github/skerryvore/regression_analysis_101/blob/master/notebooks/makgenyene_isochron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Isochron example: the Makgenyene kimberlite, South Africa

The code in this notebook demonstrates analysis of Rb-Sr isotopic data and fitting an iscochron to data for mineral separates and whole rock samples from the Makgenyene kimberlite in South Africa (Brown et al. 1989, https://doi.org/10.1016/0168-9622(89)90015-8).

![image](https://docs.google.com/uc?export=download&id=1XTF-YMinKDeOrfBdSm4QCauKhGpLy0Hd)

[//]: # "<i>Brown, R.W., Allsoppl, H.L., Bristow, J.W. and Smith, C.B., 1989. Improved precision of Rb-Sr dating of kimberlitic micas: An assessment of a leaching technique. Chemical Geology: Isotope Geoscience section, 79(2), pp.125-136.<i>"


### To run the code, run each code cell sequentially. Select the first cell (clickin the cell) and press <b>Shift-Enter</b> (Return), or if you are running the notebook on <b>Google Colab</b>, you can also just click the wee run button (black circle with a small white arrow icon, top left of cell).

### The code in the first few cells just reads the data files, and prints the first few rows of the data tables to the screen.

In [ ]:
'''
-------------------------------------------------------------------------------
Notebook example of fitting an isochron to Rb-Sr data, using the dataset from the
Makgenyene kimberlite, South Africa.

Data are published in;

Brown, R.W., Allsoppl, H.L., Bristow, J.W. and Smith, C.B., 1989. Improved precision
of Rb-Sr dating of kimberlitic micas: An assessment of a leaching technique.
Chemical Geology: Isotope Geoscience section, 79(2), pp.125-136.

The isochron is fitted and ages calculated using ordinary least squares (OLS) and
orthogonal direction regression (ODR), which takes account of errors in both the
x and y values.

The data set has been hard-coded here as python dictionaries so that the notebook is
selfcontained, i.e. no data file is required. A more elegant way to deal with the data is
of course to read the data from an Excel or csv file. Code to do this is included,
but commented out below.

------------------------------------------------------- Roderick Brown  24/10/2021
'''
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# setup default font sizes for plot elements
TINY_SIZE = 12
SMALL_SIZE = 14
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
plt.rc('figure', figsize=(10, 8))        # default figure/plot size, inches

#plt.rcParams['figure.figsize'] = (10, 8)

# Read csv files via link to github files online (needs internet)
phl_df = pd.read_csv('https://raw.githubusercontent.com/skerryvore/regression_analysis_101/master/data/makgenyene_phlogopite.csv')
wrk_df = pd.read_csv('https://raw.githubusercontent.com/skerryvore/regression_analysis_101/master/data/makgenyene_wholerock.csv')
cbn_df = pd.read_csv('https://raw.githubusercontent.com/skerryvore/regression_analysis_101/master/data/makgenyene_carbonate.csv')

# Print top 20 rows of phlogopite data table    
phl_df.head(20)

In [ ]:
# Print top 2 rows of wholerock data table 
wrk_df.head(2)


In [ ]:
# Print top 4 rows of carbonate sample data table 
cbn_df.head(5)

### Code in the next cell will plot a graph of 87Sr/86Sr versus 1/Sr concentration as a way of inspecting/evaluating Sr contamination from carbonate. Carbonate typically has very high Sr, because Sr behaves chemically like Ca, and so samples with low background Sr values show the effect of contamination most. In this case the potential carbonate contamonation is from crustal carbonate which has high 87Sr/86Sr of c. 0.71+


In [ ]:
# Setup dictionaries of values for default plotting styles
phlog_style = {'marker':'o','s':50,'color':'k','label':'phlogopites'}
whrk_style  = {'marker':'s','s':50,'color':'r','label':'whole rock'}
carbn_style = {'marker':'d','s':50,'color':'g','label':'carbonates'}

# Plot 87Sr/86Sr versus 1/total Sr
fig1, ax1 = plt.subplots() 

ax1.set_title("$^{87}Sr\,/\,^{86}Sr$ versus $1/Sr\;[ppm]$\n")
ax1.set_ylabel("$^{87}Sr\,/\,^{86}Sr$")
ax1.set_xlabel("$1/Sr\;[ppm]$")

ax1.scatter(1/np.array(phl_df['Sr [ppm]']), np.array(phl_df['87Sr/86Sr']), **phlog_style)
ax1.scatter(1/np.array(wrk_df['Sr [ppm]']), np.array(wrk_df['87Sr/86Sr']), **whrk_style)
ax1.scatter(1/np.array(cbn_df['Sr [ppm]']), np.array(cbn_df['87Sr/86Sr']), **carbn_style)

ax1.legend(loc='upper left')

plt.show(fig1)



### Plot an isochron diagram for the data, and evaluate whether a straight line fit is likely to produce a sensible age?

In [ ]:
# Plot isochron diagram, 87Sr/86Sr on y-axis versus 87Rb/86Sr on the x-axis

fig2, ax1 = plt.subplots() 

ax1.set_title("Isochron diagram\n")
ax1.set_ylabel("$^{87}Sr\,/\,^{86}Sr$")
ax1.set_xlabel("$^{87}Rb\,/\,^{86}Sr$")
        
# Plot each data set
ax1.scatter(np.array(phl_df['87Rb/86Sr']), np.array(phl_df['87Sr/86Sr']), **phlog_style)
ax1.scatter(np.array(wrk_df['87Rb/86Sr']), np.array(wrk_df['87Sr/86Sr']), **whrk_style)
ax1.scatter(np.array(cbn_df['87Rb/86Sr']), np.array(cbn_df['87Sr/86Sr']), **carbn_style)

# Draw the legend for the plot
ax1.legend(loc='lower right')

plt.show()

### Let's have a closer look at the carbonate and whole rock data by zooming into the isochron diagram...

In [ ]:
# Plot zoomed in isochron plot to examine carbonate and whole rock data

fig5, ax1 = plt.subplots() 

# Set xlimits and ylimits on plot to zoom in to low values...
ax1.set_xlim(-1,15)
ax1.set_ylim(0.7,0.73)

ax1.set_title("Focus on carbonate and whole rock data\n")
ax1.set_ylabel("$^{87}Sr\,/\,^{86}Sr$")
ax1.set_xlabel("$^{87}Rb\,/\,^{86}Sr$")
        
# Plot each data set
ax1.scatter(np.array(phl_df['87Rb/86Sr']), np.array(phl_df['87Sr/86Sr']), **phlog_style)
ax1.scatter(np.array(wrk_df['87Rb/86Sr']), np.array(wrk_df['87Sr/86Sr']), **whrk_style)
ax1.scatter(np.array(cbn_df['87Rb/86Sr']), np.array(cbn_df['87Sr/86Sr']), **carbn_style)

# Draw the legend for the plot
ax1.legend(loc='lower right')

plt.show()


### To fit a straight line, i.e. an isochron, model to the data plotted above we can use a simple linear regression approach which will estimate the "best fit" line and return the slope and intercept values for the fitted line. The code below calls the scipy Ordinary Least Squares routine called <b>stats.linregress</b> and writes some useful values to the screen.

In [ ]:
# Fit isochron using ordinary least squares regression

from scipy import stats

# Create some 1D arrays called x,y, xerr, and yerr for simplicity
x = np.array(phl_df['87Rb/86Sr'])
y = np.array(phl_df['87Sr/86Sr'])
xerr = np.array(phl_df['Rb/Sr_error'])
yerr = np.array(phl_df['1_sigma'])

# Call the scipy.stats.linregress routine with x and y arrays (defined above) as arguments
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

# Print some of the returned values of interest
print (f'OLS slope: {slope:10.8f} +/- {std_err:10.8f}')
print (f'OLS intercept: {intercept:08.6f}')
print (f'R^2 value: {r_value**2:06.4f}')
print (f'p value: {p_value:08.6f}')


### We can use the estimated slope value for the line fitted using least squares regression to calculate the age of the kimblerlite. 

### To calculate the age, $t$ after we determine the slope of the isochron line we can use this equation;

\begin{align}
 t = \ln \left (\frac{slope+1}{\lambda}\right)
\end{align}

### where $\ln$ means the natural logarithm, and $\lambda$ is the decay constant for $^{87}\mathrm{Rb}$.


In [ ]:
# Calculate the age in My using the slope
lamda87Rb = 1.419E-11 # decay constant for 87Rb in 1/yr

age = (np.log(slope +1)/lamda87Rb)/1.E6 # Divide by 1.E6 to convert to Ma

# Calculate the error on the age from the estimated std_err on the slope
relative_err = std_err/slope   # calculate the relative error
age_err_ols = age*relative_err # calculate the absolute error on the age

print (f'Age: {age:06.2f} +/- {age_err_ols:3.2f} Ma')


### And now let's plot the fitted isochron model line and the data together on an isochron diagram.

In [ ]:
# Plot the isochron diagram and the fitted line, and print the useful values

fig3, ax1 = plt.subplots() 

ax1.set_title("Isochron fitted using Ordinary Least Squares\n")
ax1.set_ylabel("$^{87}Sr\,/\,^{86}Sr$")
ax1.set_xlabel("$^{87}Rb\,/\,^{86}Sr$")

# Plot the data points
ax1.scatter(np.array(phl_df['87Rb/86Sr']), np.array(phl_df['87Sr/86Sr']), **phlog_style)
ax1.scatter(np.array(wrk_df['87Rb/86Sr']), np.array(wrk_df['87Sr/86Sr']), **whrk_style)
ax1.scatter(np.array(cbn_df['87Rb/86Sr']), np.array(cbn_df['87Sr/86Sr']), **carbn_style)

# Plot the fitted isochron line
ax1.plot(x,((x*slope) + intercept), 'r-',label='OLS regression line') # Plot the fitted line

ax1.legend(loc='lower right')

# Write the stats for the fitted line on the graph
ax1.text(5.,1.05,\
f'slope: {slope:10.8f} +/- {std_err:10.8f}\n\
intercept: {intercept:08.6f}\n \
R^2 value: {r_value**2:8.4f}\n \
p value: {p_value:04.2f}\n \
Age: {age:06.2f} +/- {age_err_ols:3.2f} Ma',fontsize='14')

# Write plot to pdf file. Change file extension to *.png or *.tif as required)
#plt.savefig('makgenyene_isochron.pdf', format='pdf')

# Show the plot on screen
plt.show()


### Note that there is one sample point that lies significantly above the isochron line, it is sample 64-B3 (with a 87Rb/86Sr value of c. 100 and an 87Sr/86Sr value of c. 0.9). This is one of the samples that was etched severley and may be indicating it has lost Rb? Let's rerun the analysis excluding this sample, and see if it makes a difference. The code in the next cell will make a copy of the phlogopite data, but will exclude sample 64-B3. 

In [ ]:
# Copy the phlogopite dataframe, but exclude the row for sample 64-B3
phl_df_new = phl_df[(phl_df.Sample != '64-B3')]

excluded_df = phl_df[(phl_df.Sample == '64-B3')] # Save a copy of 64-B3 to plot separately

phl_df_new.head(6)
                     

### Now let's rerun the analysis, and plot the new isochron diagram and look at the estimated age and associated errors. Is this new age estimate different?

In [ ]:
# Create new 1D arrays called x,y, xerr, and yerr for simplicity
x = np.array(phl_df_new['87Rb/86Sr'])
y = np.array(phl_df_new['87Sr/86Sr'])
xerr = np.array(phl_df_new['Rb/Sr_error'])
yerr = np.array(phl_df_new['1_sigma'])

# Call the scipy.stats.linregress routine with x and y arrays (defined above) as arguments
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

# Print some of the returned values of interest
print ('New values excluding sample 64-B3\n--------------------------------')
print (f'OLS slope: {slope:10.8f} +/- {std_err:10.8f}')
print (f'OLS intercept: {intercept:08.6f}')
print (f'R^2 value: {r_value**2:06.4f}')
print (f'p value: {p_value:08.6f}')

age = (np.log(slope +1)/lamda87Rb) /1.E6

# Calculate the error on the age from the estimated std_err on the slope
relative_err = std_err/slope   # calculate the relative error
age_err_ols = age*relative_err # calculate the absolute error on the age

print (f'Age: {age:06.2f} +/- {age_err_ols:3.2f} Ma')

# Plot the isochron diagram and the fitted line, and print the useful values

fig31, ax1 = plt.subplots() 

ax1.set_ylabel("$^{87}Sr\,/\,^{86}Sr$")
ax1.set_xlabel("$^{87}Rb\,/\,^{86}Sr$")

# Plot the data points
ax1.scatter(np.array(phl_df_new['87Rb/86Sr']), np.array(phl_df_new['87Sr/86Sr']), **phlog_style)
ax1.scatter(np.array(excluded_df['87Rb/86Sr']), np.array(excluded_df['87Sr/86Sr']), color='red', s=60, label='excluded samples')

ax1.legend(fontsize='x-large',loc='lower right')

# Plot the fitted isochron line
plt.plot(x,((x*slope) + intercept), 'r-',label='OLS regression line') # Plot the fitted line

# Write the stats for the fitted line on the graph
plt.text(5.,1.05,\
f'slope: {slope:10.8f} +/- {std_err:10.8f}\n\
intercept: {intercept:08.6f}\n \
R value: {r_value**2:8.4f}\n \
p value: {p_value:04.2f}\n \
Age: {age:06.2f} +/- {age_err_ols:3.2f} Ma\n\n',fontsize='14')

# Write plot to pdf file. Change file extension to *.png or *.tif as required)
#plt.savefig('makgenyene_isochron.pdf', format='pdf')

# Show the plot on screen
plt.show()


### The problem with simple ordinary least squares regression is that it does not account for the measurement uncertainties on the measured data, and the estimated standard errpr on the slope is only based on the variance from a perfect straight line. There are other regression methods we can use that DO take account of the measurement uncertainties on the data, and the fitted line and the estimated uncertainties take account of these explicitly. 

### The code in the next cell applies Orthogonal Direction Regression (ODR) to fit the isochron and reports the slope, initial ratio and the age all with estimated uncertainties. These model parameters, and their respective uncertainties, are based on the data and the uncertainties on the data.

In [ ]:
import scipy.odr.odrpack as odrpack

# Scipy's ODR, Orthogonal Distance Regression library (errors on x, y data included)
#-----------------------------------------------------------

def linear(B,x): # linear model with array values B[0] as slope and B[1] as intercept
    return(B[0]*x + B[1])

# Do ODR regression
#---------------------------------------------------------------------
# Instantiate a model object using the linear model defined above
model = odrpack.Model(linear)

# Instantiate a real data object to hold the observed data
data = odrpack.RealData(x, y, sx=xerr, sy=yerr)

# Pass data object and spherical model object to ODR solver
odrsolver = odrpack.ODR(data, model, beta0=[1.,1.])

# Run the ODR object and store output in myoutput#
myoutput = odrsolver.run()

# Print summary of output of ODR to stdout (i.e. screen)
#myoutput.pprint()

# Extract some useful stats and store them in named variables to use later
slope_odr=myoutput.beta[0]
err_slope_odr=myoutput.sd_beta[0]
intercept_odr=myoutput.beta[1]
err_intercept_odr=myoutput.sd_beta[1]
                   
# Print essential parameters and error estimates to stdout, i.e. screen
print ('\nODR regression results\n---------------------------------------')
print (f'slope: {slope_odr:08.6f}  +/-  {err_slope_odr:10.6f}')
print (f'intercept: {intercept_odr:08.6f}  +/- {err_intercept_odr:10.6f}\n')

# Calculate the age using the ODR slope
age_odr = (np.log(slope_odr +1)/lamda87Rb) /1.E6

# Calculate the error on the age from the estimated std_err on the slope
relative_err = err_slope_odr/slope_odr
age_err_odr = age*relative_err

print (f'ODR Age: {age_odr:06.2f} +/- {age_err_odr:3.2f} Ma')



### Let's plot a summmary isochron diagram showing both models, the Ordinary Least Squares (OLS) model and the Orthogonal Direction Regression (ODR) model. In ths plot the error bars (showing the measurement uncertainties on the data) are also plotted. These are so small that the symbol size of most data points obscures these, but there is one data point whic has a large uncertainty on the 87Sr/86Sr value that is clearly visible.

### Are the ages estimated using the different approaches different? Think about the estimates AND their estimated errors (these errors are at the 1 sigma level).

In [ ]:
# Plot the isochron diagram and the fitted line, and print the useful values

fig4, ax1 = plt.subplots() 

ax1.set_ylabel("$^{87}Sr\,/\,^{86}Sr$")
ax1.set_xlabel("$^{87}Rb\,/\,^{86}Sr$")

# Place all kwargs for xy data plot style in a dictionary, arguably tidier and easier to read/edit
style_data = {'marker':'o','markersize':6, 'markeredgecolor':'k',\
              'elinewidth':1.5,'linewidth':0, 'ecolor':'gray'}
ax1.errorbar(x, y, yerr, xerr, **style_data, markerfacecolor='w',label='Sample data')

# Plot ODR isochron
# Place all kwargs for plot style in a dictionary. tidier and easier to read/edit
style_odr={'color':'blue','linewidth':2,'linestyle':'-','label':'ODR isochron'}
style_ols={'color':'red','linewidth':2,'linestyle':'-','label':'OLS isochron'}

# Plot the fitted isochron lines
ax1.plot(x,((x*slope) + intercept), **style_ols) # Plot OLS  fitted line
ax1.plot(x,(slope_odr*x+intercept_odr), **style_odr) # Plot ODR  fitted line

# Write the stats for the fitted line on the graph
ax1.text(5.,1.08,\
f'OLS Age: {age:06.2f} +/- {age_err_ols:3.2f} Ma \n\n\
ODR Age: {age_odr:06.2f} +/- {age_err_odr:3.2f} Ma',fontsize='20')

ax1.legend(loc='lower right',fontsize='x-large')

# Write plot to pdf file. Change file extension to *.png or *.tif as required)
#plt.savefig('makgenyene_isochrons.pdf', format='pdf')

plt.show()
